# BITS F312 - Neural Network and Fuzzy Logic



# NNFL Assignment 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL/Data_A2/

/content/drive/MyDrive/NNFL/Data_A2


In [ ]:
# listing datasets
%ls -l

total 1234719
-rw------- 1 root root     637638 Oct 31 04:55 Assignment2.pdf
-rw------- 1 root root        259 Oct 31 04:57 class_label.mat
-rw------- 1 root root      40295 Oct 31 04:57 data55.xlsx
-rw------- 1 root root      21269 Oct 31 04:55 data5.xlsx
-rw------- 1 root root 1263647365 Oct 31 04:58 input.mat
drwx------ 2 root root       4096 Nov 18 05:58 logs/


In [ ]:
# libraries required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
# supressing warnings
import warnings
warnings.filterwarnings('ignore')

# Q1 
Implement non-linear perceptron algorithm for the classification using Online Learning (Hebbian learning)
algorithm. The dataset (data55.xlsx) contains 19 features and the last column is the output (class label).
You can use hold-out cross-validation (70, 10, and 20%) for the selection of training, validation and test
instances. Evaluate accuracy, sensitivity and specificity measures for the evaluation of test instances
(Packages such as Scikitlearn, keras, tensorflow, pytorch etc. are not allowed).

In [ ]:
def sigmoid(x):
  """
  Returns (float) sigmoid of the input variable x
  """
  val = 1/(1+ np.exp(-x))
  return val

In [ ]:
def sigmoidDerivative(x):
  """
  Returns (float) the derivative of the sigmoid of the input variable x
  """
  val =  x * (1 - x)
  return val

In [ ]:
def perceptron(X_train_data, Y_train_data, bias, W, alpha = 0.001, epochs = 20000):
  
  for i in range(epochs):

    layer = np.dot(X_train_data, W)
    input = layer + bias
    output = sigmoid(input)

    error = output - Y_train_data
    derivative = sigmoidDerivative(output)
    update = error*derivative
    WNew = np.dot(X_train_data.T, update)
    W = W - alpha*WNew
    update_bias = update
    bias = bias - alpha*update

  return W, bias
def pred_eval(X, W, bias):
  layer = np.dot(X, W)
  input = layer + bias[0]
  output = sigmoid(input)

  return output


In [ ]:
def resultQ1(filename = 'data55.xlsx'):
  dataset = pd.read_excel(filename, header = None)

  row, col = dataset.shape
  feats = col - 1 

  # normalization
  dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)
  
  # spliting dataset into train test and val
  training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

  training_data = np.array(training_data)
  validation_data = np.array(validation_data)
  testing_data = np.array(testing_data)
  training_data_X = training_data[:, :feats]
  training_data_y = training_data[:, feats]
  validation_data_X = validation_data[:, :feats]
  validation_data_y = validation_data[:, feats]
  testing_data_X = testing_data[:, :feats]
  testing_data_y = testing_data[:, feats]

  train_row, train_col = training_data_X.shape

  W = np.random.randn(train_col)  
  bias = np.ones(train_row)

  W, bias = perceptron(training_data_X, training_data_y, bias, W)
  print("The Weights after training is as follows: \n")
  pprint(W)
  print("The Bias after training is as follows: ", bias[0])

  train_pred = pred_eval(training_data_X, W, bias)
  train_pred = np.where(train_pred > 0.475, 1,0)
  print("Training Accuracy: ", (np.abs(np.sum(train_pred == training_data_y))/len(training_data_y)))

  test_pred = pred_eval(testing_data_X, W, bias)
  test_pred = np.where(test_pred > 0.475, 1,0)
  print("Testing Accuracy: ", (np.abs(np.sum(test_pred == testing_data_y))/len(testing_data_y)))
  
  validation_pred = pred_eval(validation_data_X, W, bias)
  validation_pred = np.where(validation_pred > 0.475, 1,0)
  print("Validation Accuracy: ", (np.abs(np.sum(validation_pred == validation_data_y))/len(validation_data_y)))


### Output

In [ ]:
resultQ1()

The Weights after training is as follows: 

array([ 0.41205131, -0.33590672, -0.43932103,  1.56798797,  0.21070115,
       -0.13992397, -0.13812113,  0.03820567,  1.06120044,  0.15026506,
        0.9207862 ,  0.73246924, -0.02641763,  0.62917204, -1.44522532,
        0.45782587,  0.02331333, -1.22551068,  1.00836779])
The Bias after training is as follows:  1.3100516253516938
Training Accuracy:  0.9378549098309535
Testing Accuracy:  0.8647588581456906
Validation Accuracy:  0.9673190278858435
